## Import libraries

In [ ]:
import geoai
import os

In [ ]:
cur_dir = os.path.abspath(os.path.dirname("."))

## Download sample data

We will download a sample image from Hugging Face Hub to use for car detection. You can find more high-resolution images from [OpenAerialMap](https://openaerialmap.org).

In [ ]:

raster_path = os.path.join(cur_dir, "..", "res", "naip_train.tif")
output_path = os.path.join(cur_dir, "..", "test_outputs")
os.makedirs(output_path)

cars_masks_path = os.path.join(output_path, "cars_masks.tif")
cars_json_output = os.path.join(output_path, "cars.geojson")

## Initialize the model

In [ ]:
detector = geoai.CarDetector()

## Extract cars

Extract cars from the image using the model and save the output image.

In [ ]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path=cars_masks_path,
    confidence_threshold=0.3,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(400, 400),
)

Convert the image masks to polygons and save the output GeoJSON file.

In [ ]:
gdf = detector.vectorize_masks(
    masks_path=cars_masks_path,
    output_path=cars_json_output,
    min_object_area=100,
    max_object_area=2000,
)

## Add geometric properties

In [ ]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize initial results

In [ ]:
geoai.view_vector_interactive(gdf, column="confidence")

## Filter cars by area

In [ ]:
gdf_filter = gdf[
    (gdf["area_m2"] > 8) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [ ]:
len(gdf_filter)

## Visualiza final results

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="confidence")

In [ ]:
geoai.view_vector_interactive(gdf_filter)

![image](https://github.com/user-attachments/assets/a1e4c871-b152-466a-b902-7c00e62e5444)